# Data Mining Project - Data Preparation

### Imports and Datasets

In [9]:
import pandas as pd
import data_preparation as dp
from toolz.functoolz import pipe

In [10]:
account = pd.read_csv('../data/account.csv',delimiter=';')
card_dev = pd.read_csv('../data/card_dev.csv',delimiter=';')
client = pd.read_csv('../data/client.csv',delimiter=';')
disp = pd.read_csv('../data/disp.csv',delimiter=';')
district = pd.read_csv('../data/district.csv',delimiter=';')
loan_dev = pd.read_csv('../data/loan_dev.csv',delimiter=';')
trans_dev = pd.read_csv('../data/trans_dev.csv',delimiter=';', dtype={'bank': str})

In [11]:
district = dp.calculate_average_unemployment_rate(district)
district = dp.calculate_average_commited_crimes(district)

The mean for the unemployment rate and the total number of offenses committed are calculated in the following two cells of code.

Any missing value is set to the value of the column whose value is present, e.g. if the value for the "unemploymant rate '95" is not present in a certain row, it is set to the "unemploymant rate '96" value.

The reasoning behind this strategy was that, when taken into account over the course of just one year, neither the unemployment rate nor the number of crimes would be significant. 

**ASSEMBLE MAIN_DF**

In [12]:
# join account, loan, disposition and client
main_df = account.merge(loan_dev, on='account_id', suffixes=('','_loan'), how='right')
main_df = main_df.merge(disp, on='account_id', suffixes=('','_disp'), how='left')
main_df = main_df.merge(client, on='client_id',suffixes=('','_client'), how='left')

main_df.drop(columns='district_id', axis=1, inplace=True)

# create age_at_loan and gender column
main_df = main_df.apply(lambda row: dp.calculate_age_loan(row), axis=1)

# join demograph
district.rename(columns={'code ':'code'}, inplace=True)
main_df = main_df.merge(district, left_on='district_id_client', right_on='code', how='left')

# join creditcard
main_df = main_df.merge(card_dev, on='disp_id', suffixes=('', '_card'), how='left')

In [13]:
main_df = pipe(main_df,
               dp.calculate_number_of_disponents,
               dp.calculate_diff_salary_loan,
               dp.drop_irrelevant_columns_from_main_df,
               dp.rename_main_df_columns,
               dp.drop_duplicated_accounts)

main_df.to_csv('main_df.csv', ';')

**ASSEMBLE TRANSACTIONS_DF**

In [14]:
transactions_df = account.merge(loan_dev, on='account_id', suffixes=('','_loan'), how='right').merge(trans_dev, on='account_id', suffixes=('', '_transaction'), how='left')

In [15]:
transactions_df = pipe(transactions_df,
                       dp.calculate_transaction_count,
                       dp.calculate_credit_debit_ratio,
                       dp.drop_irrelevant_columns_from_transactions_df,
                       dp.rename_transactions_df_columns)

transactions_df.to_csv('transactions_df.csv', ';')

- assessment of dimensions of data quality
- (cleaning): redundancy
- (cleaning): missing data
- (cleaning): outliers
- data transformation for compatibility with algorithms
- feature engineering from tabular data
- sampling for domain-specific purposes
- sampling for development
- imbalanced data
- feature selection

##### **Redundancy**

explain what has been done regarding redundant data

##### **Missing Data**

explain what has been done regarding missing data

##### **Outliers**

explain what has been done regarding outliers

##### **Other data preparation operations**

explain what has been done additionally